In [1]:
!pip install arxiv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 KB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for sgmllib3k, since package 'wheel' is not installed.
  Running setup.py install for sgmllib3k ... done


Wikipedia Agent Code Here 

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [15]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [16]:
wiki.name

'wikipedia'

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb=Chroma.from_documents(documents, OllamaEmbeddings(model="llama3.2:1b"))
retriever=vectordb.as_retriever()
retriever


/tmp/ipykernel_645151/3050913996.py:9: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vectordb=Chroma.from_documents(documents, OllamaEmbeddings(model="llama3.2:1b"))


VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x72e5ac109750>, search_kwargs={})

In [9]:
from langchain.tools.retriever import create_retriever_tool

retrieval_tool=create_retriever_tool(retriever,"langsmith_search",
                      "search for the infromation about the Langsmit. for any qustion about LangSmith.")

In [11]:
retrieval_tool.name

'langsmith_search'

Arxiv Agent Code Here

In [12]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun



In [14]:
arxiv_api_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv_tool=ArxivQueryRun(arxiv_api_wrapper=arxiv_api_wrapper)

In [24]:
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=os.getenv("GOOGLE_API_KEY"))



In [ ]:
### Alpha Vantage Test
# from langchain_community.utilities import AlphaVantageAPIWrapper
from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["ALPHA_VANTAGE_API_KEY"] = os.getenv("ALPHA_VANTAGE_API_KEY")

alpha_vantage = AlphaVantageAPIWrapper(alphavantage_api_key=os.getenv("ALPHA_VANTAGE_API_KEY"))
# alpha_vantage._get_exchange_rate("USD", "INR")  # Example: Convert USD to EUR


# alpha_vantage._get_quote_endpoint("IBM")

{'Global Quote': {'01. symbol': 'IBM',
  '02. open': '242.7400',
  '03. high': '250.8900',
  '04. low': '242.4900',
  '05. price': '248.6600',
  '06. volume': '6794972',
  '07. latest trading day': '2025-03-31',
  '08. previous close': '244.0000',
  '09. change': '4.6600',
  '10. change percent': '1.9098%'}}

In [55]:
from langchain.tools import Tool

def get_exchange_rate(from_currency: str, to_currency: str) -> str:
    """Fetch the exchange rate from Alpha Vantage."""
    exchange_rate_info = alpha_vantage.run(from_currency, to_currency)
    return exchange_rate_info

# Create the tool
exchange_rate_tool = Tool(
    name="Exchange Rate Tool",
    func=get_exchange_rate,
    description="Fetches the exchange rate between two currencies using Alpha Vantage."
)


In [57]:
tools=[wiki,arxiv_tool,retrieval_tool,exchange_rate_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/dev/Desktop/Learning/AI/project1/env/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='langsmith_search', description='search for the infromation about the Langsmit. for any qustion about LangSmith.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x72e5ae11a0e0>, retriever=VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vecto

In [35]:
from langchain import hub
from langchain.agents import create_react_agent



prompt = hub.pull("hwchase17/react")
prompt

### Agents



agent=create_react_agent(llm,tools,prompt)

In [38]:
from langchain.agents import AgentExecutor

agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith_search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'Chroma\', \'OllamaEmbeddings\'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x72e5ac109750>, search

In [39]:
agent_executor.invoke({"input":"What is LangSmith?"})



> Entering new AgentExecutor chain...
LangSmith is a tool related to LangChain. I should use the `langsmith_search` tool to find out what LangSmith is.
Action: langsmith_search
Action Input: "What is LangSmith?"LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you co

{'input': 'What is LangSmith?',
 'output': "LangSmith is a platform that provides observability, evaluation, and prompt engineering tools for building and improving applications that use large language models (LLMs). It can be used with or without LangChain's open-source frameworks. It helps in debugging, monitoring, and optimizing LLM applications throughout their development stages."}

In [59]:
agent_executor.invoke({"input":"What is the exchange rate from USD to INR?"})



> Entering new AgentExecutor chain...
I need to find the current exchange rate between USD and INR. I don't have a tool that provides real-time financial data. I will try Wikipedia to see if it provides a general overview or links to resources that might have the information.
Action: wikipedia
Action Input: USD to INR exchange ratePage: Exchange rate history of the Indian rupee
Summary: This is a list of tables showing the historical timeline of the exchange rate for the Indian rupee (INR) against the special drawing rights uniWikipedia didn't provide the current exchange rate. It only provided the exchange rate history. I don't have access to a real-time financial data tool. Therefore, I cannot answer the question.
Final Answer: I am sorry, I cannot provide the current exchange rate for USD to INR. My tools do not have access to real-time financial data.

> Finished chain.


{'input': 'What is the exchange rate from USD to INR?',
 'output': 'I am sorry, I cannot provide the current exchange rate for USD to INR. My tools do not have access to real-time financial data.'}